# Базовые понятия статистики

In [241]:
import pandas as pd
import numpy as np

In [344]:
df = pd.read_csv(
    'horse_data.csv', usecols = [0,1,3,4,5,6,10,22],
    names = ['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome'],
    na_values = "?")
df.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


### выбросы по столбцам

- По лекции не было понятно, как находить выбросы, По документу с лекции  не вышло. Применила метод, найденный в интернете

In [353]:
cols = df.columns 

print('Выбросы в столбцах: ')
for i in range(len(cols)):
    ba = df[cols[i]].quantile(0.99)
    if ba < 9:
        print(f'\t\t\t{cols[i]}: {ba}')
    

Выбросы в столбцах: 
			surgery?: 2.0
			temperature of extremities: 4.0
			pain: 5.0
			outcome: 3.0


### базовые статистики

In [287]:
cols = df.columns 

for i in range(len(cols)):
	print(f'Столбец "{cols[i]}"\n\
    Минимальное значение - {df[cols[i]].min()}\n\
    Максимальное значение - {df[cols[i]].max()}\n\
    Медиана - {df[cols[i]].median()}\n\
    Среднее арифметическое - {df[cols[i]].mean()}\n\
    Мода - {df[cols[i]].round().mode()[0]}\n')


Столбец "surgery?"
    Минимальное значение - 1.0
    Максимальное значение - 2.0
    Медиана - 1.0
    Среднее арифметическое - 1.3979933110367893
    Мода - 1.0

Столбец "Age"
    Минимальное значение - 1
    Максимальное значение - 9
    Медиана - 1.0
    Среднее арифметическое - 1.64
    Мода - 1

Столбец "rectal temperature"
    Минимальное значение - 35.4
    Максимальное значение - 40.8
    Медиана - 38.2
    Среднее арифметическое - 38.16791666666669
    Мода - 38.0

Столбец "pulse"
    Минимальное значение - 30.0
    Максимальное значение - 184.0
    Медиана - 64.0
    Среднее арифметическое - 71.91304347826087
    Мода - 48.0

Столбец "respiratory rate"
    Минимальное значение - 8.0
    Максимальное значение - 96.0
    Медиана - 24.5
    Среднее арифметическое - 30.417355371900825
    Мода - 20.0

Столбец "temperature of extremities"
    Минимальное значение - 1.0
    Максимальное значение - 4.0
    Медиана - 3.0
    Среднее арифметическое - 2.348360655737705
    Мода - 3.0


# заполнения пропусков 
### что заполняем  - В зависимости от чего, предположение
 - Ректальная температура - возраст 
 - пульс - возраст, температура ректальная 
 - Боль - хирургия 
 - Дыхание - боль 
 - температура конечностей - возраст, дыхание 

In [345]:
df['rectal temperature'] = df['rectal temperature'].fillna(df.groupby('Age')['rectal temperature'].transform('median'))  
df['pulse'] = df['pulse'].fillna(df.groupby(['Age', 'rectal temperature'])['pulse'].transform('median'))
df['pain'] = df['pain'].fillna(df.groupby('surgery?')['pain'].transform('max'))  
df['respiratory rate'] = df['respiratory rate'].fillna(df.groupby('pain')['respiratory rate'].transform('median'))  
df['temperature of extremities'] = df['temperature of extremities'].fillna(df.groupby(['respiratory rate', 'Age'])['temperature of extremities'].transform('median'))


df.head(10)

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,2.0,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,3.5,5.0,2.0
5,2.0,1,38.1,70.0,32.0,2.0,2.0,1.0
6,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
7,1.0,1,38.1,60.0,28.0,3.0,5.0,2.0
8,2.0,1,38.1,80.0,36.0,3.0,4.0,3.0
9,2.0,9,38.3,90.0,28.0,1.0,5.0,1.0
